# Model Registry in SageMaker
This notebook demonstrates how to register a trained model in the SageMaker Model Registry. 
It includes steps for creating a model package, adding metadata, and managing model versions.

So in this notebook, we will use the model we trained in the previous notebook, `[model_training.ipynb](./model_training.ipynb)`.   

## About the Model Registry
The SageMaker Model Registry is a centralized repository for managing machine learning models. It allows you to:
- **Register Models**: Store and manage trained models with versioning.
- **Track Model Metadata**: Maintain metadata such as model performance metrics, training data, and environment configurations.
- **Promote Models**: Move models through different stages (e.g., development, staging, production) using lifecycle policies.

### Step 1: Import Required Libraries & Set Up Environment

In [ ]:
# Import necessary libraries
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import time
from sagemaker.session import Session
from sagemaker import Model
from sagemaker import ModelPackage

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define the S3 bucket and prefix for storing data
s3_bucket = sagemaker_session.default_bucket()  # Default bucket created by SageMaker
s3_prefix = 'emp-bonus-linearlearner'  # Prefix for organizing data in S3

print(f"Using bucket: {s3_bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Using bucket: sagemaker-us-east-1-767397738123


### Step 2: Prepare Model

In [2]:
# Upload the model artifact to S3
local_file_path = 'data/model.tar.gz'
s3_uri = sagemaker_session.upload_data(local_file_path, bucket=s3_bucket, key_prefix = s3_prefix)
print(f"File uploaded to: {s3_uri}")
model_artifact = f's3://{s3_bucket}/{s3_prefix}/model.tar.gz'

# Determine what AWS region we are in
region = sagemaker_session.boto_region_name
# Retrieve the Linear Learner image URI
container_image_uri = sagemaker.image_uris.retrieve(framework="linear-learner", region=region)

print(model_artifact)

# Print path to container image
print(f"SageMaker Linear Learner Image URI: {container_image_uri}")

# Assign a meaningful name to the model
model_name = 'emp-bonus'  # Give it a name for identification

# Create the SageMaker model
model = Model(
    image_uri=container_image_uri,  # Use the appropriate image URI for your model's container
    model_data=model_artifact,      # Path to the model artifact (model.tgz)
    role=get_execution_role(),
    sagemaker_session=sagemaker_session,
    name=model_name
)

File uploaded to: s3://sagemaker-us-east-1-767397738123/emp-bonus-linearlearner/model.tar.gz
s3://sagemaker-us-east-1-767397738123/emp-bonus-linearlearner/model.tar.gz
SageMaker Linear Learner Image URI: 382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


### Step 3: Create a Model Package Group in SageMaker Model Registry

In [3]:
# Create a model package group
model_package_group_name = 'emp-bonus-prediction-group'
model_package_group_description = 'SageMaker model package group for employee bonus prediction'
sagemaker_client = boto3.client('sagemaker')

try:
    sagemaker_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription=model_package_group_description
    )
    print(f'Created Model Package Group: {model_package_group_name}')
    
except sagemaker_client.exceptions.ResourceInUse:
    print(f'Model Package Group {model_package_group_name} already exists.')


Created Model Package Group: emp-bonus-prediction-group


### Step 4: Register Model into SageMaker Model Registry

In [12]:
# Register the model

model_approval_status = "PendingManualApproval"
customer_metadata_properties = {"ModelType": "EmpBonusPrediction"}

print(model_approval_status)
print(model_package_group_name)
print(customer_metadata_properties)
model_package = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    customer_metadata_properties=customer_metadata_properties,
)

print(f"Model package version ARN: {model_package.model_package_arn}")

PendingManualApproval
emp-bonus-prediction-group
{'ModelType': 'EmpBonusPrediction'}
Model package version ARN: arn:aws:sagemaker:us-east-1:767397738123:model-package/emp-bonus-prediction-group/3


### Step 5: Retrieve the Model from the SageMaker Model Registry

In [14]:
# Fetch the latest approved model package from the model registry
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    ModelApprovalStatus='Approved',
    SortBy='CreationTime',
    SortOrder='Descending'
)

latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
print(f'Latest Approved Model Package ARN: {latest_model_package_arn}')

# Fetch all model packages in the model group (any status)
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name,
    SortBy='CreationTime',
    SortOrder='Descending'
)

print("\nAll versions with their status:\n")

# Print all model package ARNs and their approval status
for model_pkg in response['ModelPackageSummaryList']:
    arn = model_pkg['ModelPackageArn']
    status = model_pkg.get('ModelApprovalStatus', 'N/A')
    print(f"ARN: {arn} | Status: {status}")


Latest Approved Model Package ARN: arn:aws:sagemaker:us-east-1:767397738123:model-package/emp-bonus-prediction-group/1

All versions with their status:

ARN: arn:aws:sagemaker:us-east-1:767397738123:model-package/emp-bonus-prediction-group/3 | Status: Rejected
ARN: arn:aws:sagemaker:us-east-1:767397738123:model-package/emp-bonus-prediction-group/2 | Status: PendingManualApproval
ARN: arn:aws:sagemaker:us-east-1:767397738123:model-package/emp-bonus-prediction-group/1 | Status: Approved
